In [1]:
# https://simplekml.readthedocs.io/en/latest/


In [2]:
import math
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import simplekml as sk
from polycircles import polycircles
import subprocess
from latlon import LatLon, Latitude, Longitude
import math

In [4]:
def set_color(colour):
    """color, icon_ref = set_color(colour)"""
    if colour == 'blue':
        color=sk.Color.blue
        icon_ref = 'http://maps.google.com/mapfiles/kml/paddle/blu-blank.png'

    if colour == 'lime':
        color=sk.Color.lime
        icon_ref = 'http://maps.google.com/mapfiles/kml/paddle/ltblu-diamond.png'
    elif colour == 'purple':
        color=sk.Color.purple
        icon_ref = 'http://maps.google.com/mapfiles/kml/paddle/purple-diamond.png'
    elif colour == 'red':
        color=sk.Color.red
        icon_ref = 'http://maps.google.com/mapfiles/kml/paddle/red-diamond.png'
    elif colour == 'green':
        color=sk.Color.green
        icon_ref = 'http://maps.google.com/mapfiles/kml/paddle/grn-diamond.png'
    else:
        color=sk.Color.white
        icon_ref = 'http://maps.google.com/mapfiles/kml/shapes/mechanic.png'
    return color, icon_ref

In [5]:
def kml_circle(name, kml, latitude=-37.280344, longitude=174.652676, altitude=120, radius=10000, number_of_vertices=60, label_at=0, colour='purple'):
    polycircle = polycircles.Polycircle(latitude, longitude, radius, number_of_vertices)
    coords = polycircle.to_kml()
    # append_circle
    coords = [(c[0], c[1], altitude) for c in coords]
    # cir_kml = sk.Kml()
    color, icon_ref = set_color(colour)
    circle = kml.newlinestring(name=name)
    circle.coords = coords
    circle.style.polystyle.color = sk.Color.changealphaint(100, color)
    circle.polystyle.outline = 0
    circle.linestyle.color = sk.Color.changealphaint(200, color)
    circle.linestyle.width = 2
    circle.altitudemode = sk.AltitudeMode.relativetoground
    circle.extrude = 1
    pnt = kml.newpoint(name=name)
    pnt.coords = [coords[label_at]]
    pnt.style.labelstyle.color = color  # Make the text red
    pnt.style.labelstyle.scale = 1
    pnt.altitudemode = sk.AltitudeMode.relativetoground
    pnt.style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/purple-diamond.png'

kml = sk.Kml()
kml_circle('C2 Link', kml, radius=30000, altitude=5000, label_at=50)
kml.save("../kml/C2Link.kml", format=0)

kml.newpoint()

In [9]:
def append_circle(coords=None, radius=1, number_of_vertices=60, clockwise=True, semicircle=False):
    if coords is None:
        coords = [[0, 0, 0]]
    if not clockwise:
        clockwise = -1
    if semicircle == True:
        circumference = 1*math.pi*radius
        degrees = 180.0
    else:
        circumference = 2*math.pi*radius
        degrees = 360.0
    segment = circumference/number_of_vertices
    heading = coords[0][0]
    for i in range(number_of_vertices+1):
        degree = heading + (clockwise*degrees/number_of_vertices)*i
        coords.append([degree, segment])
    # coords.append(coords[0])
    return coords

print (append_circle([(90,0,0)], 60/(2*math.pi), 6, False))
60/(2*math.pi)


[(90, 0, 0), [90.0, 10.0], [30.0, 10.0], [-30.0, 10.0], [-90.0, 10.0], [-150.0, 10.0], [-210.0, 10.0], [-270.0, 10.0]]


9.549296585513721

In [10]:
def kml_circle2(name, kml=None, coords=None, latitude=-37.280344, longitude=174.652676, radius=20, heading=0, altitude=500, label_at=None, colour='red'):
    if kml is None:
        kml = sk.Kml()
    if coords is None:
        coords = [[heading-90, 0, 0],]
    coords = append_circle(coords, radius, number_of_vertices=60, clockwise=True, semicircle=True)
    coords.append([heading-180, radius*2, 0])
    coords = [[c[0], c[1], altitude] for c in coords]

    color, icon_ref = set_color(colour)
    # assert coords[0][:2] == [0,0], 'The first coord must be 0,0,0'
    _coords = [None]*len(coords)
    dist = 0
    start = LatLon(Latitude(latitude), Longitude(longitude))
    start = start.offset(heading-180, radius)
    for i, c in enumerate(coords):
        _coords[i] = [start.offset(c[0], c[1])]
        dist += c[1]
        start = _coords[i][0]
    _coords = [(_c[0].lon.decimal_degree, _c[0].lat.decimal_degree, c[2]) for _c, c in zip(_coords, coords)]
    if label_at is None:
        label_at = len(_coords)//2
    cir  = kml.newlinestring(name=name)
    cir.coords = _coords
    cir.style.polystyle.color = sk.Color.changealphaint(100, color)
    cir.polystyle.outline = 0
    cir.linestyle.color = sk.Color.changealphaint(200, color)
    cir.linestyle.width = 3
    cir.altitudemode = sk.AltitudeMode.relativetoground
    cir.extrude = 1
    pnt = kml.newpoint(name=name)
    pnt.coords = [_coords[label_at]]
    pnt.style.labelstyle.color = color # Make the text red
    pnt.style.labelstyle.scale = 1
    pnt.altitudemode = sk.AltitudeMode.relativetoground
    pnt.style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/red-diamond.png'
    return kml, dist

kml, dist = kml_circle2('Circle Path', latitude=-37.280344, longitude=174.652676, coords=None,  heading=-23, label_at=-3,)
print('Total Flightpath distance = ', dist )
kml.save("../kml/circle.kml", format=0)


Total Flightpath distance =  103.87905062299241


In [11]:
latitude=-37.291465
longitude=174.653884


In [12]:
def kml_flight_Path(name, kml=None, coords=None, latitude=-37.280344, longitude=174.652676, heading=0, scale=1, label_at=None, colour='green'):
    if kml is None:
        kml = sk.Kml()
    if coords is None:
        coords = [(0, 0, 0), (0, 1, 0), (-90, 1, 0), (180, 1, 0), (90, 1, 0), ]

    assert coords[0][:2] == [0,0], 'The first coord must be 0,0'
    _coords = [None]*len(coords)
    color, icon_ref = set_color(colour)

    dist = 0
    start = LatLon(Latitude(latitude), Longitude(longitude))
    for i, c in enumerate(coords):
        _coords[i] = [start.offset(c[0]+heading, c[1]*scale)]
        dist += c[1]*scale
        start = _coords[i][0]
    _coords = [(_c[0].lon.decimal_degree, _c[0].lat.decimal_degree, c[2]) for _c, c in zip(_coords, coords)]
    if label_at is None:
        label_at = len(_coords)//2
    f_path  = kml.newlinestring(name=name)
    f_path.coords = _coords
    f_path.style.polystyle.color = sk.Color.changealphaint(100, color)
    f_path.polystyle.outline = 0
    f_path.linestyle.color = sk.Color.changealphaint(200, color)
    f_path.linestyle.width = 3
    f_path.altitudemode = sk.AltitudeMode.relativetoground
    f_path.extrude = 1
    pnt = kml.newpoint(name=name)
    pnt.coords = [_coords[label_at]]
    pnt.style.labelstyle.color = color # Make the text red
    pnt.style.labelstyle.scale = 1
    pnt.altitudemode = sk.AltitudeMode.relativetoground
    pnt.style.iconstyle.icon.href = icon_ref
    return kml, dist

flight_path = [[0, 0],  [-90, 1], [0, 0]]
flight_path = append_circle(flight_path, 1, 10, clockwise=False)
flight_path += [[0, 10], [-90, 0.5], ]
for i in range(10):
    flight_path += [[180, 20], [-90, 0.5], [0, 20], [-90, 0.5],]
flight_path += [[180, 10-0.1], [90, 0.5*20+0.25],[90, 1], ]

flight_path = [ [f[0], f[1], 120] for f in flight_path]
flight_path[0][2] = 0
flight_path[-1][2] = 0

kml, dist = kml_flight_Path('Flight Path', latitude=-37.280344, longitude=174.652676, coords=flight_path,  heading=-23, label_at=-3,colour='red')
print('Total Flightpath distance = ', dist )
kml.save("../kml/FlightPath2.kml", format=0)

Total Flightpath distance =  449.56150383789753


In [13]:
def kml_launch_Area(name, kml=None, coords=None, latitude=-37.280344, longitude=174.652676, heading=0, scale=1, label_at=None):
    if kml is None:
        kml = sk.Kml()
    if coords is None:
        coords = [[0, 0, 50], [0, 0.75, 50], [-90, 1, 50], [180, 1, 50], [90, 1, 50], [0, 0.25, 50]]

    assert coords[0][:2] == [0,0], 'The first coord must be 0,0,0'
    _coords = [None]*len(coords)
    color=sk.Color.yellow
    dist = 0
    start = LatLon(Latitude(latitude), Longitude(longitude))
    for i, c in enumerate(coords):
        _coords[i] = [start.offset(c[0]+heading, c[1]*scale)]
        dist += c[1]*scale
        start = _coords[i][0]
    _coords = [(_c[0].lon.decimal_degree, _c[0].lat.decimal_degree, c[2]) for _c, c in zip(_coords, coords)]
    if label_at is None:
        label_at = len(_coords)//2
    pol  = kml.newpolygon(name='')
    pol.outerboundaryis = _coords
    pol.style.polystyle.color = sk.Color.changealphaint(100, color)
    pol.polystyle.outline = 0
    pol.linestyle.color = sk.Color.changealphaint(200, color)
    pol.linestyle.width = 3
    pol.altitudemode = sk.AltitudeMode.relativetoground
    pol.extrude = 1
    pnt = kml.newpoint(name=name)
    pnt.coords = [_coords[label_at]]
    pnt.style.labelstyle.color = color # Make the text red
    pnt.style.labelstyle.scale = 1
    pnt.altitudemode = sk.AltitudeMode.relativetoground
    pnt.style.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/paddle/ylw-diamond.png'
    return kml

kml= kml_launch_Area('launch_area', latitude=-37.280344, longitude=174.652676,  heading=-23, scale=0.25, label_at=-2,)

kml.save("../kml/LaunchArea2.kml", format=0)

In [14]:
def kml_Area(name, kml=None, coords=None, latitude=-37.280344, longitude=174.652676, heading=0, scale=1, label_at=None, colour='blue'):
    if kml is None:
        kml = sk.Kml()
    if coords is None:
        coords = [[-90, 0.85, 50], [0, 13, 50], [-90, 15, 50], [180, 26, 50], [90, 15, 50], [0, 15, 50]]

    _coords = [None]*len(coords)

    color, icon_ref = set_color(colour)
    dist = 0
    start = LatLon(Latitude(latitude), Longitude(longitude))
    for i, c in enumerate(coords):
        _coords[i] = [start.offset(c[0]+heading, c[1]*scale)]
        dist += c[1]*scale
        start = _coords[i][0]

    _coords[-1] = _coords[0]
    _coords = [(_c[0].lon.decimal_degree, _c[0].lat.decimal_degree, c[2]) for _c, c in zip(_coords, coords)]

    if label_at is None:
        label_at = len(_coords)//2
    path  = kml.newlinestring(name=name)
    path.coords = _coords
    path.style.polystyle.color = sk.Color.changealphaint(100, color)
    path.polystyle.outline = 0
    path.linestyle.color = sk.Color.changealphaint(200, color)
    path.linestyle.width = 3
    path.altitudemode = sk.AltitudeMode.relativetoground
    path.extrude = 1
    pnt = kml.newpoint(name=name)
    pnt.coords = [_coords[label_at]]
    pnt.style.labelstyle.color = color # Make the text red
    pnt.style.labelstyle.scale = 1
    pnt.altitudemode = sk.AltitudeMode.relativetoground
    pnt.style.iconstyle.icon.href = icon_ref
    return kml

In [15]:
latitude=-37.280344
longitude=174.652676
coords = [[-90, 0.85, 50], [0, 13, 50], [-90, 15, 50], [180, 26, 50], [90, 15, 50], [0, 15, 50]]
kml= kml_Area('opertions_area', kml=None, coords=coords, latitude=latitude, longitude=longitude, heading=-23, scale=1, label_at=2, colour='blue')

coords = [[-90, 0.7, 50], [0, 13.15, 50], [-90, 15.3, 50], [180, 26.3, 50], [90, 15.3, 50], [0, 15, 50]]
kml= kml_Area('1-1 keepout area', kml=kml, coords=coords, latitude=latitude, longitude=longitude, heading=-23, scale=1, label_at=3, colour='red')
kml= kml_launch_Area('launch_area', kml=kml, latitude=latitude, longitude=longitude, heading=-23, scale=0.25, label_at=-2,)
kml_circle('C2 Link',  kml=kml, latitude=latitude, longitude=longitude, radius=30000, altitude=5000, label_at=50)

flight_path = [[0, 0],  [-90, 1], [0, 0]]
flight_path = append_circle(flight_path, 1, 10, clockwise=False)
flight_path += [[0, 10], [-90, 0.5], ]
for i in range(10):
    flight_path += [[180, 20], [-90, 0.5], [0, 20], [-90, 0.5],]
flight_path += [[180, 10.5], [90, 0.5*20+0.25],[90, 1], ]

flight_path = [ [f[0], f[1], 120] for f in flight_path]
flight_path[0][2] = 0
flight_path[-1][2] = 0

kml, dist = kml_flight_Path('Flight Path Offshore Wind', kml=kml, latitude=-37.280344, longitude=174.652676, coords=flight_path,  heading=-23, label_at=-3,)
flight_path[1][1] = 2
flight_path[-1][1] = 2

kml, dist = kml_flight_Path('Flight Path Onshore Wind', kml=kml, latitude=-37.280344, longitude=174.652676, coords=flight_path,  heading=-23, label_at=-3,colour='lime')
print('Total Flightpath distance = ', dist )
kml.save("../kml/FlightPlan2.kml", format=0)

Total Flightpath distance =  452.16150383789756


In [16]:

import simplekml
kml = simplekml.Kml()
pnt = kml.newpoint(name='Kariotahi', coords=[(174.653884, -37.281465)])
pnt.camera.latitude = -37.291465
pnt.camera.longitude = 174.653884
pnt.camera.altitude = 10000
pnt.camera.tilt = 45
pnt.camera.heading = 0
pnt.camera.roll = 0
pnt.camera.altitudemode = simplekml.AltitudeMode.relativetoground
# pnt.save("../kml/Camera.kml")
kml.save("../kml/Camera2.kml")